## Introduction
This kernel is just a simple sample code that "classifies" whether input images are whale or dolphin.

### Input train.csv

In [ ]:
import os
import pandas as pd

data_root = '/kaggle/input/happy-whale-and-dolphin'
df = pd.read_csv(os.path.join(data_root, 'train.csv'))

### Fix Meta Data

In [ ]:
df['species'] = df['species'].str.replace('beluga', 'beluga_whale')
df['species'] = df['species'].str.replace('globis', 'globis_whale')
df['species'] = df['species'].str.replace('bottlenose_dolpin', 'bottlenose_dolphin')
df['species'] = df['species'].str.replace('kiler_whale', 'killer_whale')

### Make Dataset

In [ ]:
import tensorflow as tf

def load_and_preprocess_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [128, 128])
    img /= 255.0
    return img

img_paths = list(map(lambda x: os.path.join(data_root, 'train_images', x), df['image']))
path_ds = tf.data.Dataset.from_tensor_slices(img_paths)
img_ds = path_ds.map(load_and_preprocess_img)
labels = list(map(lambda x: 0 if 'whale' in x else 1, df['species']))
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(labels, tf.int64))
ds = tf.data.Dataset.zip((img_ds, label_ds))
ds = ds.shuffle(100).repeat(1).batch(32)

### Model

In [ ]:
from tensorflow.keras import Model, layers, applications

class MyResNet50(Model):
    def __init__(self):
        super().__init__()
        self.base_model = applications.ResNet50(include_top=False, pooling='max')
        self.fc1 = layers.Dense(1024, activation='relu')
        self.fc2 = layers.Dense(2, activation='softmax')

    def call(self, x):
        x = self.base_model(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

### Training

In [ ]:
model = MyResNet50()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(ds, epochs=1)